# Multi-Step MLP Model

references :

- How to Develop Multilayer Perceptron Models for Time Series Forecasting

  https://machinelearningmastery.com/how-to-develop-multilayer-perceptron-models-for-time-series-forecasting/
  


- Exploratory Configuration of a Multilayer Perceptron Network for Time Series Forecasting 

  https://machinelearningmastery.com/exploratory-configuration-multilayer-perceptron-network-time-series-forecasting/  


# Web traffic case

In [ ]:
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from math import sqrt
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

##### read dataset

In [ ]:
wt_df = read_csv('train_2.csv', nrows=1)
wt_df

##### select and transform dataframe row to sequence

The dataframe is converted to sequence or array format 

In [ ]:
# take only the values without the page name

def get_seq_by_row(input_df, row_num):
    new_seq = np.delete(input_df.iloc[row_num].values, 0)
    return new_seq

wt_seq = get_seq_by_row(wt_df,0)

In [ ]:
wt_seq

#### time lag parameter

The following are the first 2 hyperparameter:
    - time_step_lag : the number of NN input values  
    - time_step_ahead : the number of NN output values, which means the number of days to predict

In [ ]:
time_step_lag = 7 # input of NN
time_step_ahead = 30 # number of day to be predicted

#### split train and test set

The series is splitted into train and test set. 

The size of the train set is the original size subtracted by the time_step_head. The values are taken from the first values of the original series with the previouly mentioned size. The format of the train set is still a sequence.

The test set is output in format of 2-dimention array with only one array entry, which holds the test set values. The values contains the input and the output values for the neural network and the size respectively is the same as time_step_lag and time_step_ahead. The input values are taken from the last values of the train set with already mentioned size and the output values are the remaining values of the original series after taking out the train set.


In [ ]:
def split_train_and_test_set(data_seq, time_step_lag, time_step_ahead):
    train_seq = data_seq[:-time_step_ahead]
    test_set = np.array([data_seq[-(time_step_lag + time_step_ahead):]])
    return train_seq, test_set

In [ ]:
train_seq, test = split_train_and_test_set(wt_seq, time_step_lag, time_step_ahead)

In [ ]:
train_seq

In [ ]:
test

##### transform sequence to supervised format

the train set is converted into multiple sequences or array with each array has the size of combination of time_step_lag and time_step_ahead. 

Each sequence is formed by moving window scanning from first values of the train set series. 

The moving window has the size of the required array size and it shifts by 1 to form the next array. 

In [ ]:
# frame a sequence as a supervised learning problem

def timeseries_to_supervised(data, lag=1, stepahead=1):
    df = DataFrame(data)
    col_num = lag+stepahead
    columns = [df.shift(i) for i in range(1, col_num)]
    columns = list(reversed(columns))
    columns.append(df)
    df = concat(columns, axis=1)    
    return df.values[col_num - 1:,:]

In [ ]:
wt_supervised = timeseries_to_supervised(train_seq, time_step_lag, time_step_ahead)
wt_supervised

#### split train and validation set

From the reformated train set above, validation set is built by subtracting n number of array from the last entries

In [ ]:
def split_train_and_validation_set(dataset, num_of_test_set=12):
    num_of_test_set = -1 * num_of_test_set
    return dataset[0:num_of_test_set], dataset[num_of_test_set:]    

In [ ]:
train, validation = split_train_and_validation_set(wt_supervised)

In [ ]:
validation

#### scale sequence value

All the values are scaled into the range of -1 to 1. 

The scaler is built by fitting the train set on each value index or each variable.

The scaler is then used to scale validation set values and test set values.

In [ ]:
# scale train and test data to [-1, 1]

def scale(train, test):
    
    # fit scaler
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(train)

    # transform train
    train = train.reshape(train.shape[0], train.shape[1])
    train_scaled = scaler.transform(train)
    
    # transform test
    test = test.reshape(test.shape[0], test.shape[1])
    test_scaled = scaler.transform(test)
    
    return scaler, train_scaled, test_scaled

def scale_with_scaler(scaler, data):
    
    # transform data
    data = data.reshape(data.shape[0], data.shape[1])
    data_scaled = scaler.transform(data)
    
    return data_scaled

In [ ]:
scaler, train_scaled, validation_scaled = scale(train, validation)

In [ ]:
validation_scaled

In [ ]:
test_scaled = scale_with_scaler(scaler, test)

In [ ]:
test_scaled

#### build model

the model has only one hidden layer with variable number of neuron and uses relu as activation function.

the output layer uses the value of time_step_ahead to determine the number of output values.

the loss function measures the MSE to calculate the error and optimizes the parameter using the adam optimizer.

the model is packed in a function with input parameter of train set, batch size, number of epoch, neurons, time_step_ahead

In [ ]:
# fit an MLP network to training data

def fit_model(train, batch_size, nb_epoch, neurons, time_step_ahead):
    
    X, y = train[:, 0:-time_step_ahead], train[:, -time_step_ahead:]

    model = Sequential()
    
    # hidden layer
    model.add(Dense(neurons, activation='relu', input_dim=X.shape[1]))
    
    # output layer
    model.add(Dense(time_step_ahead))
    
    # loss function
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    # model fitting
    #model.fit(X, y, epochs=nb_epoch, batch_size=batch_size, verbose=0, shuffle=False)
    model.fit(X, y, epochs=nb_epoch, verbose=0, shuffle=False)
    
    return model

#### fit model 

setting the hyperparameter of the NN model and then training the model 

In [ ]:
batch_size = 4
epochs = 1000
neurons = 3

In [ ]:
model = fit_model(train_scaled, batch_size, epochs, neurons, time_step_ahead)

#### make forecasting

prediction is done by calling the evaluate method, which uses the fitted model to predict the given input values

In [ ]:
def evaluate(model, scaled_input, time_step_ahead):
    
    eval_input = scaled_input[:,0:-time_step_ahead]
    
    output = model.predict(eval_input)
    
    return output

In [ ]:
train_output = evaluate(model, train_scaled, time_step_ahead)
train_output

In [ ]:
validation_output = evaluate(model, validation_scaled, time_step_ahead)
validation_output

In [ ]:
test_output = evaluate(model, test_scaled, time_step_ahead)
test_output

#### invert scale the prediction

using the scaler above to invert back all the predicted values.

the predicted values must be set together with the input values in order to be inverted.

In [ ]:
# inverse scaling for a forecasted value

def invert_scale(scaler, X, yhat):    
    new_row = [x for x in X] + [x for x in yhat]    
    array = np.array(new_row)
    array = array.reshape(1, len(array))
    inverted = scaler.inverse_transform(array)
    
    return inverted[0, -len(yhat):]

def invert_scale_prediction(scaler, scaled_set, scaled_output):
    scaled_input = scaled_set[:,0:-time_step_ahead]
    predictions = list()

    for i in range(len(scaled_output)):
        yhat = scaled_output[i]
        X = scaled_input[i]

        # invert scaling
        yhat = invert_scale(scaler, X, yhat)    

        # store forecast
        predictions.append(yhat)
    
    return predictions

In [ ]:
train_unscaled_output = invert_scale_prediction(scaler, train_scaled, train_output)
train_unscaled_output

In [ ]:
validation_unscaled_output = invert_scale_prediction(scaler, validation_scaled, validation_output)
validation_unscaled_output

In [ ]:
test_unscaled_output = invert_scale_prediction(scaler, test_scaled, test_output)
test_unscaled_output

#### calculate root mean squared error

In [ ]:
def calculate_rmse(original, prediction, time_step_ahead):
    
    test_output = original[:,-time_step_ahead:]
    rmse = sqrt(mean_squared_error(test_output, prediction))
    
    return rmse

In [ ]:
train_rmse = calculate_rmse(train, train_unscaled_output, time_step_ahead)

print('Train RMSE: %.3f' % (train_rmse))

In [ ]:
validation_rmse = calculate_rmse(validation, validation_unscaled_output, time_step_ahead)

print('Validation RMSE: %.3f' % (validation_rmse))

In [ ]:
test_rmse = calculate_rmse(test, test_unscaled_output, time_step_ahead)

print('Test RMSE: %.3f' % (test_rmse))

#### vary the time step lag

In [ ]:
# config

# time_step_lag = 2
time_step_lag_array = np.arange(45,51)

time_step_ahead = 30



batch_size = 4
epochs = 2000
neurons = 20



train_rmse_array = []
validation_rmse_array = []

for time_step_lag in time_step_lag_array:

    # split train - test set
    train_seq, test = split_train_and_test_set(wt_seq, time_step_lag, time_step_ahead)
    
    # tranform data to NN input format
    wt_supervised = timeseries_to_supervised(train_seq, time_step_lag, time_step_ahead)

    # split train and test set
    train, validation = split_train_and_validation_set(wt_supervised)

    # scale dataset
    scaler, train_scaled, validation_scaled = scale(train, validation)
    
    # fit model
    model = fit_model(train_scaled, batch_size, epochs, neurons, time_step_ahead)

    # evaluate train set
    train_output = evaluate(model, train_scaled, time_step_ahead)
    train_unscaled_output = invert_scale_prediction(scaler, train_scaled, train_output)
    train_rmse = calculate_rmse(train, train_unscaled_output, time_step_ahead)
    train_rmse_array.append(train_rmse)
    
    # evaluate test set
    validation_output = evaluate(model, validation_scaled, time_step_ahead)
    validation_unscaled_output = invert_scale_prediction(scaler, validation_scaled, validation_output)
    validation_rmse = calculate_rmse(validation, validation_unscaled_output, time_step_ahead)
    validation_rmse_array.append(validation_rmse)
    
    print('%d) TrainRMSE=%f, ValidationRMSE=%f' % (time_step_lag, train_rmse, validation_rmse))

In [ ]:
print([round(x,2) for x in train_rmse_array])
print([round(x,2) for x in validation_rmse_array])

#### plot RMSE

In [ ]:
%matplotlib notebook

plt.ylabel('RMSE')
plt.xlabel('time step lag')
plt.plot(time_step_lag_array, train_rmse_array, '-', linewidth=1, color='orange', label='train RMSE')
plt.plot(time_step_lag_array, validation_rmse_array, '-', linewidth=1, color='blue', label='validation RMSE')  
plt.legend(loc='right')
plt.show()

#### vary the hidden layer neuron

In [ ]:
# config

time_step_lag = 26
# time_step_lag_array = np.arange(20,26)

time_step_ahead = 30

# split train - test set
train_seq, test = split_train_and_test_set(wt_seq, time_step_lag, time_step_ahead)



batch_size = 4
epochs = 2000

# neurons = 20
neurons_array = np.arange(20,26)



train_rmse_array = []
validation_rmse_array = []

for neurons in neurons_array:

    # tranform data to NN input format
    wt_supervised = timeseries_to_supervised(train_seq, time_step_lag, time_step_ahead)

    # split train and test set
    train, validation = split_train_and_validation_set(wt_supervised)

    # scale dataset
    scaler, train_scaled, validation_scaled = scale(train, validation)
    
    # fit model
    model = fit_model(train_scaled, batch_size, epochs, neurons, time_step_ahead)

    # evaluate train set
    train_output = evaluate(model, train_scaled, time_step_ahead)
    train_unscaled_output = invert_scale_prediction(scaler, train_scaled, train_output)
    train_rmse = calculate_rmse(train, train_unscaled_output, time_step_ahead)
    train_rmse_array.append(train_rmse)
    
    # evaluate test set
    validation_output = evaluate(model, validation_scaled, time_step_ahead)
    validation_unscaled_output = invert_scale_prediction(scaler, validation_scaled, validation_output)
    validation_rmse = calculate_rmse(validation, validation_unscaled_output, time_step_ahead)
    validation_rmse_array.append(validation_rmse)
    
    print('%d) TrainRMSE=%f, ValidationRMSE=%f' % (neurons, train_rmse, validation_rmse))    

#### plot rmse

In [ ]:
%matplotlib notebook

plt.ylabel('RMSE')
plt.xlabel('time step lag')
plt.plot(time_step_lag_array, train_rmse_array, '-', linewidth=1, color='orange', label='train RMSE')
plt.plot(time_step_lag_array, validation_rmse_array, '-', linewidth=1, color='blue', label='validation RMSE')  
plt.legend(loc='right')
plt.show()

## THE MAIN PART OF BUILDING NN MODEL FOR COMPARING WITH ARIMA
## Using smoothed dataset from R 

the following code is the main part to build model and calculate the prediction, which then is compared to the ARIMA model.

However, this will require the defined methods above to be executed first.

#### read dataset from R

In R the dataset is already smoothed using moving average.

The dataset in R is exported to CSV File and read here.

In [ ]:
wt_df = read_csv('data.csv')
wt_df

#### take only the moving average 30 values

the values smoothed by moving average with frequency of 30 are going to be used

In [ ]:
wt_seq = wt_df['Clicks.MA30'].dropna().values
wt_seq

#### optimizing model by the number of time step lag

- time step lag value is varied to find the best value for time_step_lag 
- the procedures are
    - initialzing the parameter
    - loop through the all values of time_step_lag
    - in the loop
        - train and test set are spitted
        - the train set is converted to input-output array format
        - the train set is splitted into train and validation set format
        - the values are scaled
        - the model is fitted
        - the prediction is made
        - the values are inverted back

In [ ]:
# config

#time_step_lag = 26
time_step_lag_array = np.arange(5,81,5)

time_step_ahead = 30



batch_size = 4
epochs = 2000

neurons = 25
#neurons_array = np.arange(25,30)



train_rmse_array = []
validation_rmse_array = []

for time_step_lag in time_step_lag_array:
    
    # split into train - test set
    train_seq, test = split_train_and_test_set(wt_seq, time_step_lag, time_step_ahead)

    # tranform data to NN input format
    wt_supervised = timeseries_to_supervised(train_seq, time_step_lag, time_step_ahead)

    # split train and test set
    train, validation = split_train_and_validation_set(wt_supervised)

    # scale dataset
    scaler, train_scaled, validation_scaled = scale(train, validation)
    
    # fit model
    model = fit_model(train_scaled, batch_size, epochs, neurons, time_step_ahead)

    # evaluate train set
    train_output = evaluate(model, train_scaled, time_step_ahead)
    train_unscaled_output = invert_scale_prediction(scaler, train_scaled, train_output)
    train_rmse = calculate_rmse(train, train_unscaled_output, time_step_ahead)
    train_rmse_array.append(train_rmse)
    
    # evaluate test set
    validation_output = evaluate(model, validation_scaled, time_step_ahead)
    validation_unscaled_output = invert_scale_prediction(scaler, validation_scaled, validation_output)
    validation_rmse = calculate_rmse(validation, validation_unscaled_output, time_step_ahead)
    validation_rmse_array.append(validation_rmse)
    
    print('%d) TrainRMSE=%f, ValidationRMSE=%f' % (time_step_lag, train_rmse, validation_rmse))

#### save the validation result in csv for time step lag

In [ ]:
validation_res = pd.DataFrame({'time_step_lag': time_step_lag_array, 'validationRMSE': validation_rmse_array})
validation_res.to_csv('../ml2/data/validation-time-step-lag.csv')
validation_res

#### optimizing model by the number of neurons

- the number of neuron is varied to find the best number of neuron
- the procedures are
    - initialzing the parameter
    - train and test set are spitted
    - loop through the all values of number of neuron    
    - in the loop
        - the train set is converted to input-output array format
        - the train set is splitted into train and validation set format
        - the values are scaled
        - the model is fitted
        - the prediction is made
        - the values are inverted back

In [ ]:
# config

time_step_lag = 50

time_step_ahead = 30

# split train - test set
train_seq, test = split_train_and_test_set(wt_seq, time_step_lag, time_step_ahead)



batch_size = 4
epochs = 2000

# neurons = 20
neurons_array = np.arange(5,51,5)



train_rmse_array = []
validation_rmse_array = []

for neurons in neurons_array:

    # tranform data to NN input format
    wt_supervised = timeseries_to_supervised(train_seq, time_step_lag, time_step_ahead)

    # split train and test set
    train, validation = split_train_and_validation_set(wt_supervised)

    # scale dataset
    scaler, train_scaled, validation_scaled = scale(train, validation)
    
    # fit model
    model = fit_model(train_scaled, batch_size, epochs, neurons, time_step_ahead)

    # evaluate train set
    train_output = evaluate(model, train_scaled, time_step_ahead)
    train_unscaled_output = invert_scale_prediction(scaler, train_scaled, train_output)
    train_rmse = calculate_rmse(train, train_unscaled_output, time_step_ahead)
    train_rmse_array.append(train_rmse)
    
    # evaluate test set
    validation_output = evaluate(model, validation_scaled, time_step_ahead)
    validation_unscaled_output = invert_scale_prediction(scaler, validation_scaled, validation_output)
    validation_rmse = calculate_rmse(validation, validation_unscaled_output, time_step_ahead)
    validation_rmse_array.append(validation_rmse)
    
    print('%d) TrainRMSE=%f, ValidationRMSE=%f' % (neurons, train_rmse, validation_rmse))    

#### save the validation result in csv for neurons

In [ ]:
validation_res = pd.DataFrame({'neurons': neurons_array, 'validationRMSE': validation_rmse_array})
validation_res.to_csv('../ml2/data/validation-neuron.csv')
validation_res

#### build model with best parameter

the best parameters based on the above grid search are use to build model.

the model is fitted using the train set including the validation set.

In [ ]:
time_step_lag = 50

time_step_ahead = 30

# split into train - test set
train_seq, test = split_train_and_test_set(wt_seq, time_step_lag, time_step_ahead)



batch_size = 4
epochs = 2000
neurons = 45

# tranform data to NN input format
wt_supervised = timeseries_to_supervised(train_seq, time_step_lag, time_step_ahead)

# split train and test set
train, validation = split_train_and_validation_set(wt_supervised)

# scale dataset
scaler, train_scaled, validation_scaled = scale(wt_supervised, validation)

# fit model
model = fit_model(train_scaled, batch_size, epochs, neurons, time_step_ahead)


#### evaluate test set

In [ ]:
test_scaled = scale_with_scaler(scaler, test)
test_output = evaluate(model, test_scaled, time_step_ahead)
test_unscaled_output = invert_scale_prediction(scaler, test_scaled, test_output)
test_rmse = calculate_rmse(test, test_unscaled_output, time_step_ahead)

print('Test RMSE: %.3f' % (test_rmse))

#### save test result in csv

In [ ]:
test_res = pd.DataFrame({'actual': test[0,-time_step_ahead:], 'prediction': test_unscaled_output[0]})
test_res

In [ ]:
test_res.to_csv('../ml2/data/test-lag-50-neuron-45.csv')